In [1]:
import numpy as np
import random
import math
#from scipy.stats import multivariate_normal, norm
import derivatives as dev
from tqdm import tqdm
import matplotlib.pyplot as plt
import lambda_sampler as ls
import tau_sampler as ts
#from scipy.stats import halfcauchy

In [2]:
extracted_coefficients_directory = '../../../data/commaai/extracted_coefficients/20201027_filtered_gaussian_resampled/'
B_zeta_path = str(extracted_coefficients_directory + 'Bzeta/B_zeta.npy')
beta_path = str(extracted_coefficients_directory + 'beta/beta.csv')
z_path = str(extracted_coefficients_directory + 'Bzeta/tr_labels.npy')
beta = np.genfromtxt(beta_path, delimiter=',')
B_zeta = np.load(B_zeta_path)
B_zeta = B_zeta.reshape(B_zeta.shape[0], beta.shape[0])
z = np.load(z_path)#[0:B_zeta.shape[0]]
n = B_zeta.shape[0]
p = B_zeta.shape[1]

Lambda_old = np.repeat(1,10) #np.array([1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])# # np.array([random.uniform(0, 1) for i in range(0,10)])
BoB = B_zeta**2
log_tau_old = math.log(0.5)
dS2_old, ddS2_old, S2_old, S_old = dev.generate_dS2_ddS2_S2_S(Lambda_old, BoB)

In [3]:
# initialize Lambda
betaBt = beta.dot(B_zeta.T)

In [4]:
num_iterations = 10000
lambdaacc = np.zeros((len(Lambda_old),num_iterations))
all_betas = []
all_lambdas = []
tauacc = []
all_taus = []
for j in tqdm(range(0, num_iterations)):
    
    beta = ts.sample_beta(p, B_zeta, Lambda_old, S_old, z)
    all_betas.append(beta)
    
    betaBt = beta.dot(B_zeta.T)
    
    # update tau
    log_tau_new, decision_criterion_tau = ts.sample_tau(log_tau_old, Lambda_old, p)
    # decision to accept/reject tau
    
    if math.log(random.random()) <= decision_criterion_tau:
        log_tau_old = log_tau_new.copy()
        tauacc.append(1)
        all_taus.append(log_tau_new)
    else:
        tauacc.append(0)
        all_taus.append(log_tau_old)
    
    # update lambda
    for m in range(0, len(Lambda_old)):
        log_sq_lambda_new, decision_criterion_lambda, dS2_new, ddS2_new, S2_new, S_new, ddlunew, Lambda_new = ls.sample_lambda(m, 
                                                                                               log_tau_old, 
                                                                                               Lambda_old, p, beta, 
                                                                                               B_zeta, dS2_old, ddS2_old, 
                                                                                              S2_old, S_old, z, BoB, betaBt)
        
        if math.log(random.random()) <= decision_criterion_lambda:
            if m < p - 1:
                Lambda_old = np.append(np.append(Lambda_old[0:m], math.exp(0.5*log_sq_lambda_new)), Lambda_new[m + 1:])
            #Lambda_old = np.copy(Lambda_new)
            elif m == p - 1:
                Lambda_old = np.append(Lambda_old[0:m],  math.exp(0.5*log_sq_lambda_new))
            lambdaacc[m,j] += 1
            all_lambdas.append(Lambda_old)
            dS2_old, ddS2_old, S2_old, S_old = dS2_new.copy(), ddS2_new.copy(), S2_new.copy(), S_new.copy()
        
        else:
            lambdaacc[m,j] += 0
            all_lambdas.append(Lambda_old) 

  0%|          | 24/10000 [00:22<2:39:08,  1.04it/s]


KeyboardInterrupt: 

In [5]:
print('Acceptance rate for lambda_js: ' + str(np.sum(lambdaacc/num_iterations, axis = 1))
     + '\nAcceptance rate for tau: ' + str(np.mean(tauacc)))


Acceptance rate for lambda_js: [0.0015 0.     0.0014 0.     0.0002 0.     0.0001 0.     0.0002 0.    ]
Acceptance rate for tau: 0.96


In [ ]:
np.mean(lambdaacc[0:46], axis = 1)

In [ ]:
plt.plot(all_lambdas)

In [ ]:
all_lambdas = np.array(all_lambdas)
for i in range(0, all_lambdas.shape[1]):
    plt.plot(range(0, all_lambdas.shape[0]), all_lambdas[:,i])

In [ ]:
lam.shape

In [ ]:
lam1 = lam[:,0].reshape(279)

In [ ]:
plt.plot(range(0,279), lam1)

In [ ]:
plt.plot(np.exp(all_taus))

In [ ]:
all_taus